In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

In [2]:
#载入数据集
mnist=input_data.read_data_sets('MNIST_data',one_hot=True)

#运行次数
max_steps=1001
#图片数量
image_num=3000
#文件路径
DIR='/home/x/deeplearning/Tenworflow/Learn_TF/'

#载入图片
embedding=tf.Variable(tf.stack(mnist.test.images[:image_num]),trainable=False,name='embedding')


#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean=tf.reduce_mean(var)
        tf.summary.scalar('mean',mean) #平均值
        with tf.name_scope('stddev'):
            stddev=tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)  #标准差
        tf.summary.scalar('max',tf.reduce_max(var))  #最大值
        tf.summary.scalar('min',tf.reduce_min(var))  #最小值
        tf.summary.histogram('histogram',var)  #直方图
        

#命名空间
with tf.name_scope('input'):
    #定义placeholder
    x=tf.placeholder(tf.float32,[None,784],name='x-input')
    y=tf.placeholder(tf.float32,[None,10],name='y-input')

#显示图片
with tf.name_scope('input_reshape'):
    image_shaped_input=tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input',image_shaped_input,10)
    
with tf.name_scope('layer'):
    #创建一个简单的神经网络
    with tf.name_scope('weights'):
        Weights=tf.Variable(tf.zeros([784,10]),name='W')
        variable_summaries(Weights)
    with tf.name_scope('biases'):
        biases=tf.Variable(tf.zeros([1,10]),name='b')
        variable_summaries(biases)
    with tf.name_scope('wx_plus_b'):    
        wx_plus_b=tf.matmul(x,Weights)+biases
    with tf.name_scope('softmax'):  
        prediction=tf.nn.softmax(wx_plus_b)


#二次代价函数
# loss=tf.reduce_mean(tf.square(y-prediction))
# loss=tf.reduce_mean(-tf.reduce_sum(y*tf.log(prediction),reduction_indices=[1]))
with tf.name_scope('loss'):
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss',loss)
with tf.name_scope('train'):
    train_step=tf.train.GradientDescentOptimizer(0.1).minimize(loss)

#初始化变量
init=tf.global_variables_initializer()

with tf.name_scope('accuracy'):
    #求准确率,结果存放在布尔型列表中
    with tf.name_scope('corrent_prediction'):
        corrent_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回最大值的位置
    with tf.name_scope('accuracy'):
        accuracy=tf.reduce_mean(tf.cast(corrent_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)
        
with tf.Session() as sess:
    sess.run(init)
    
#产生metdata文件
    if tf.gfile.Exists(DIR+'projector/projector/metadata.tsv'):
        tf.gfile.DeleteRecursively(DIR+'projector/projector/metadata.tsv')
    with open (DIR+'projector/projector/metadata.tsv','w') as f:
        labels=sess.run(tf.argmax(mnist.test.labels[:],1))
        for i in range(image_num):
            f.write(str(labels[i])+'\n')
            
#合并所有summary
    merged=tf.summary.merge_all()
    
    projector_writer=tf.summary.FileWriter(DIR+'projector/projector',sess.graph)
    saver=tf.train.Saver()
    config=projector.ProjectorConfig()
    embed=config.embeddings.add()
    embed.tensor_name=embedding.name
    embed.metadata_path=DIR+'projector/projector/metadata.tsv'
    embed.sprite.image_path=DIR+'projector/data/mnist_10k_sprite.png'
    embed.sprite.single_image_dim.extend([28,28])
    projector.visualize_embeddings(projector_writer,config)
    
    for i in range(max_steps):
        #每个批次100个样本
        batch_xs,batch_ys=mnist.train.next_batch(100)
        run_options=tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata=tf.RunMetadata()
        summary,_=sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys},options=run_options,run_metadata=run_metadata)
        projector_writer.add_run_metadata(run_metadata,'step%03d' % i)
        projector_writer.add_summary(summary,i)

        if i%100==0:
            acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
            print('Iter'+str(i) +' : Testing Accuracy:'+str(acc))
            
    saver.save(sess,DIR+'projector/projector/a_model.ckpt',global_step=max_steps)
    projector_writer.close()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Iter0 : Testing Accuracy:0.3042
Iter100 : Testing Accuracy:0.5981
Iter200 : Testing Accuracy:0.7216
Iter300 :

In [5]:
#载入数据集
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
#运行次数
max_steps = 100
#图片数量
image_num = 500
#文件路径
DIR = "/home/x/deeplearning/Tenworflow/Learn_TF/"
 
#定义会话
sess = tf.Session()
 
#载入图片
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]), trainable=False, name='embedding')
 
#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)#平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)#标准差
        tf.summary.scalar('max', tf.reduce_max(var))#最大值
        tf.summary.scalar('min', tf.reduce_min(var))#最小值
        tf.summary.histogram('histogram', var)#直方图
 
#命名空间
with tf.name_scope('input'):
    #这里的none表示第一个维度可以是任意的长度
    x = tf.placeholder(tf.float32,[None,784],name='x-input')
    #正确的标签
    y = tf.placeholder(tf.float32,[None,10],name='y-input')
 
#显示图片
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)
 
with tf.name_scope('layer'):
    #创建一个简单神经网络
    with tf.name_scope('weights'):
        W = tf.Variable(tf.zeros([784,10]),name='W')
        variable_summaries(W)
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10]),name='b')
        variable_summaries(b)
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x,W) + b
    with tf.name_scope('softmax'):    
        prediction = tf.nn.softmax(wx_plus_b)
 
with tf.name_scope('loss'):
    #交叉熵代价函数
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss',loss)
with tf.name_scope('train'):
    #使用梯度下降法
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
 
#初始化变量
sess.run(tf.global_variables_initializer())
 
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        #结果存放在一个布尔型列表中
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大的值所在的位置
    with tf.name_scope('accuracy'):
        #求准确率
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))#把correct_prediction变为float32类型
        tf.summary.scalar('accuracy',accuracy)
 
#产生metadata文件
if tf.gfile.Exists(DIR + 'projector/projector/metadata.tsv'):
    tf.gfile.DeleteRecursively(DIR + 'projector/projector/metadata.tsv')
with open(DIR + 'projector/projector/metadata.tsv', 'w') as f:
    labels = sess.run(tf.argmax(mnist.test.labels[:],1))
    for i in range(image_num):   
        f.write(str(labels[i]) + '\n')        
         
#合并所有的summary
merged = tf.summary.merge_all()   
 
 
projector_writer = tf.summary.FileWriter(DIR + 'projector/projector',sess.graph)
saver = tf.train.Saver()
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = DIR + 'projector/projector/metadata.tsv'
embed.sprite.image_path = DIR + 'projector/data/mnist_10k_sprite.png'
embed.sprite.single_image_dim.extend([28,28])
projector.visualize_embeddings(projector_writer,config)
 
for i in range(max_steps):
    #每个批次100个样本
    batch_xs,batch_ys = mnist.train.next_batch(100)
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    summary,_ = sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys},options=run_options,run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata, 'step%03d' % i)
    projector_writer.add_summary(summary, i)
     
    if i%100 == 0:
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print ("Iter " + str(i) + ", Testing Accuracy= " + str(acc))
 
saver.save(sess, DIR + 'projector/projector/a_model.ckpt', global_step=max_steps)
projector_writer.close()
sess.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


InvalidArgumentError: You must feed a value for placeholder tensor 'input/y-input' with dtype float and shape [?,10]
	 [[Node: input/y-input = Placeholder[dtype=DT_FLOAT, shape=[?,10], _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: layer_1/biases/summaries/Mean/_71 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_308_layer_1/biases/summaries/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'input/y-input', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/lib/python3.6/site-packages/traitlets/